In [1]:
import pandas as pd
import numpy as np
hitters=pd.read_csv("Beyzbol/Hitters.csv")

pandas, numpy kütüphanelerini ve üzerinde çalışacağımız veri setini dahil ediyoruz.

In [2]:
def rssCalculater(dataFrame, columnName, threshold, target):
    lowerDf , higherDf =dfSplit(dataFrame,columnName,threshold)
    
    rss=0
    
    lowerMean=lowerDf[target].mean()
    rss += ((lowerDf[target]-lowerMean)**2).sum()
    
    higherMean=higherDf[target].mean()
    rss += ((higherDf[target]-higherMean)**2).sum()

    return rss

rssCalculater fonksiyonu veri setindeki herhangi bir kolon adı ve threshold için ona karşılık gelen RSS değerini hesaplıyor.

In [3]:
def dfSplit(dataFrame, columnName, threshold):
    lowerDf=dataFrame.loc[dataFrame[columnName]<threshold]
    higherDf=dataFrame.loc[dataFrame[columnName]>=threshold]
    return lowerDf,higherDf

dfSplit fonksiyonu veri setini ikiye ayrılıyor.Başka bir değişle veri setini herhangi bir kolonun belirli bir değerinden
    büyük ve küçük olanları olmak üzere ayırıyor.

In [4]:
def leafValue(dataFrame):
    value=dataFrame["Salary"].mean()
    return value

leafValue fonksiyonu yapraklardaki değerin tahmin edilen target("Salary") ortalamasını buluyor.

In [5]:
def bestSplit(dataFrame):
    minRss=0
    threshold=0
    rssColumn=str
    for column in dataFrame.select_dtypes(include=["integer"]):
        for row in dataFrame[column]:
            rss=rssCalculater(dataFrame,column,row,"Salary")
            if minRss==0:
                minRss=rss
            if rss<minRss:
                threshold=row
                minRss=rss
                rssColumn=column
    return rssColumn,threshold,minRss

bestSplit fonksiyonu bir veri setini ayırırken en optimal kolon ve threshold değerini veriyor.

In [6]:
def treeSplit(dataFrame, rssColumn, threshold, maxDepth, currentDepth=1, name="", dic={}):
    
    lowerDf , higherDf = dfSplit(dataFrame,rssColumn,threshold)
    
    controller=True
    if currentDepth>maxDepth:
        return
    
    else:
        if len(lowerDf)<=5 or len(higherDf)<=5:
            controllerColumn=rssColumn
            controllerThreshold=threshold
            controller=False
        
            
        if len(lowerDf)>5:
            if len(name) >= currentDepth:
                name=name[:-1]
            name+="L"
            rssColumn,threshold,minRss=bestSplit(lowerDf)
            if currentDepth==4:
                dic[name]=rssColumn,threshold,leafValue(lowerDf)
            else:
                dic[name]=rssColumn,threshold
            treeSplit(lowerDf,rssColumn,threshold,maxDepth,currentDepth+1,name,dic,)
        
        else:
            if len(name) > currentDepth:
                name=name[:-1]
            name+="L"
            dic[name]=rssColumn,threshold,leafValue(lowerDf)
            name=name[:-1]
            
        if len(higherDf)>5:
            if len(name) >= currentDepth:
                name=name[:-1]
            name+="R"
            rssColumn,threshold,minRss=bestSplit(higherDf)
            
            if currentDepth==4:
                dic[name]=rssColumn,threshold,leafValue(higherDf)
                
            else:
                dic[name]=rssColumn,threshold
            treeSplit(higherDf,rssColumn,threshold,maxDepth,currentDepth+1,name,dic)
        
        else:
            
            if len(name) >= currentDepth:
                name=name[:-1]    
            name+="R"
            
            if controller==False:
                dic[name]=controllerColumn,controllerThreshold,leafValue(higherDf)
                controller=True
                
            else:
                dic[name]=rssColumn,threshold,leafValue(higherDf)
                name=name[:-1]
                
    return dic

treeSplit fonksiyonu maxsize(bir yaprakta en az 5 veri bulunma) ve maxdepth(ağacın derinliği) şartları altında ağacın kurallarını kaydediyoruz. Recursive function kullanarak ağacı oluşturduğumuz kurallar bütününü dictionary tipinde kayıt ediyoruz. Ağacı oluşturduğumuz fonksiyon.

In [7]:
def buildTree(dataFrame,maxDepth):
    
    dic={}
    
    root,threshold,minRSS = bestSplit(dataFrame)
    
    dic = treeSplit(dataFrame,root,threshold,maxDepth=maxDepth)
    
    dic["root"]=root,threshold
        
    return dic

buildTree fonksiyonu ağacı bütün kurallarıyla oluşturduğumuz fonksiyon.

In [8]:
def prediction(testData,dic,maxDepth=4):

    if testData[dic["root"][0]] >= dic["root"][1]:
        name="R"
        while len(name) < maxDepth:

            if len(dic[name])==3:
                return dic[name][2]
            if testData[dic[name][0]] < dic[name][1]:
                name+="L"

            else:
                name+="R"
        return dic[name][2]
    else:
        name="L"

        while len(name)< maxDepth:

            if len(dic[name])==3:
                return dic[name][2]
                
            if testData[dic[name][0]] < dic[name][1]:
                name+="L"

            else:
                name+="R"
                    
        return dic[name][2]

prediction fonksiyonu herhangi test verisinin oluşturduğumuz ağaçtan yararlanarak alacağı tahmin edilen değeri söylüyor.

In [9]:
def generalRSS(testData, target, dic):
    
        #dic = self.buildTree(trainData)

        predictionValues=[]
        rss=0

        for i in testData.index:

            predictionValues.append((prediction(testData.loc[i],dic), testData.loc[i][target]))

        for i in predictionValues:

            rss += ((i[0] - i[1])**2) / len(testData.index)
        return rss

generalRSS fonksiyonu test verilerinin oluşturduğumuz ağaçta tahmin edilen değerleri ile gerçek değerleri arasındaki farktan
RSS buluyor.

In [10]:
def bestDepth(trainData,testData,target):
    rssList=[]
    for depth in range(1,6):
        dic=buildTree(trainData,depth)
        trainRSS=generalRSS(trainData, target, dic)
        testRSS=generalRSS(testData, target, dic)
        rssList.append((trainRSS, testRSS, depth))
    return rssList

In [11]:
x = hitters.dropna(subset=["Salary"])
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(x, test_size=.2, random_state=41)

In [12]:
tree=buildTree(hitters,4)
tree

{'L': ('Hits', 3),
 'LL': ('Hits', 3, 2127.333),
 'LR': ('CAtBat', 1258),
 'LRL': ('AtBat', 22),
 'LRLL': ('AtBat', 22, 920.0),
 'LRLR': ('CHits', 184, 154.97318390804597),
 'LRR': ('Walks', 47),
 'LRRL': ('Errors', 15, 312.1666666666667),
 'LRRR': ('CRBI', 198, 451.0),
 'R': ('Walks', 62),
 'RL': ('PutOuts', 1160),
 'RLL': ('AtBat', 370),
 'RLLL': ('CHmRun', 194, 490.81523913043486),
 'RLLR': ('CRBI', 280, 742.1126607142857),
 'RLR': ('PutOuts', 1160, 1712.5),
 'RR': ('RBI', 104),
 'RRL': ('CRBI', 314),
 'RRLL': ('RBI', 60, 668.8888888888889),
 'RRLR': ('Hits', 160, 1147.2988275862067),
 'RRR': ('RBI', 104, 1859.25775),
 'root': ('CHits', 452)}

In [13]:
test=hitters.loc[20]
prediction(test, tree)

154.97318390804597

In [20]:
def bagging():
    rssValues=[]
    for treeCount in range(1,5):
        x_train, x_test = train_test_split(x, train_size=0.3)
        tree=buildTree(x_test, maxDepth=4)
        RSS=generalRSS(x_train,"Salary",tree)
        rssValues.append(RSS)
    return rssValues

In [ ]:
rssValues=bagging()

In [ ]:
total=0
for i in rssValues:
    total+=i
print(total/len(a))